In [22]:
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from astropy.table import Table
from itertools import combinations
from astropy.coordinates import SkyCoord
from astropy import units as u


save = True

sns.set(style="whitegrid")

## Example

## INPUT: Gaia-style observables

You give `SkyCoord` the usual 6D astrometric quantities:

* `ra`, `dec`: right ascension and declination (angular position on the sky)
* `parallax`: gives distance → $d = \frac{1000}{\text{parallax in mas}}$
* `pmra`, `pmdec`: proper motions (tangential angular speed)
* `radial_velocity`: line-of-sight speed

---

## What SkyCoord Does Internally

### 1. **Position (x, y, z)**

It transforms your angular position + distance into Cartesian coordinates in the **ICRS (equatorial) frame**, centered on the Sun:

$$
\begin{aligned}
x &= d \cdot \cos(\text{dec}) \cdot \cos(\text{ra}) \\
y &= d \cdot \cos(\text{dec}) \cdot \sin(\text{ra}) \\
z &= d \cdot \sin(\text{dec})
\end{aligned}
$$

where `ra` and `dec` are in radians, and `d` is in parsecs.

### 2. **Velocity (vx, vy, vz)**

It uses the radial velocity and the proper motions to get the 3D space velocity:

* Proper motions are angular → converted into **km/s** using:

  $$
  v = 4.74047 \times \mu \times d
  $$
* It then projects these tangential and radial velocities into Cartesian components, aligned with the $x, y, z$ from above.

---

## Example:

If you do:

```python
c = SkyCoord(ra=..., dec=..., distance=..., pm_ra_cosdec=..., pm_dec=..., radial_velocity=...)
```

Then:

```python
c.cartesian.xyz  → gives (x, y, z) in pc  
c.velocity.d_xyz → gives (vx, vy, vz) in km/s
```

All in a consistent 3D Cartesian frame, with origin at the Sun and axes aligned with the ICRS system:

* $x$ toward the vernal equinox (RA=0)
* $z$ toward celestial north pole (Dec=+90°)
* $y$ completes the right-handed system

---


In [23]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

def compute_orbital_elements_from_phase_space_without_rv(star1, star2):
    G = 4.302e-3  # pc * (km/s)^2 / Msun

    # SkyCoord objects without radial velocity
    c1 = SkyCoord(
        ra=star1['ra'] * u.deg,
        dec=star1['dec'] * u.deg,
        distance=(1000 / star1['parallax']) * u.pc,
        pm_ra_cosdec=star1['pmra'] * u.mas/u.yr,
        pm_dec=star1['pmdec'] * u.mas/u.yr,
        radial_velocity=star1['radial_velocity'] * u.km/u.s
    )

    c2 = SkyCoord(
        ra=star2['ra'] * u.deg,
        dec=star2['dec'] * u.deg,
        distance=(1000 / star2['parallax']) * u.pc,
        pm_ra_cosdec=star2['pmra'] * u.mas/u.yr,
        pm_dec=star2['pmdec'] * u.mas/u.yr,
        radial_velocity=star2['radial_velocity'] * u.km/u.s
    )

    # Masses and CM quantities
    m1, m2 = star1['mass'], star2['mass']
    M = m1 + m2
    M_r = (m1 * m2) / M
    mu = G*(m1 * m2)

    # Kepler way:
    

    r1 = c1.cartesian.xyz.to(u.pc).value
    v1 = c1.velocity.d_xyz.to(u.km/u.s).value  # Tangential velocity
    r2 = c2.cartesian.xyz.to(u.pc).value
    v2 = c2.velocity.d_xyz.to(u.km/u.s).value  # Tangential velocity

    # Relative position and velocity in CM frame
    r_rel = r2 - r1
    v_rel = v2 - v1
    r_mag = np.linalg.norm(r_rel)
    v_mag = np.linalg.norm(v_rel)

    # Angular momentum
    h_vec = np.cross(r_rel, v_rel)
    h_mag = np.linalg.norm(h_vec)

    # Specific mechanical energy
    E = M_r*(0.5*v_mag**2. - mu/r_mag) - G*m1*m2 / r_mag
    a = - mu/(2 * E)  # Semi-major axis

    print(f"Computed energy total: {E:.3f}")

    # Eccentricity vector
    e_vec = (np.cross(v_rel, h_vec) / (G * M)) - (r_rel / r_mag)
    e = np.linalg.norm(e_vec)

    # Inclination
    i_rad = np.arccos(h_vec[2] / h_mag)
    i_deg = np.degrees(i_rad)

    # Node vector
    K = np.array([0, 0, 1])
    n_vec = np.cross(K, h_vec)
    n_mag = np.linalg.norm(n_vec)

    # Longitude of ascending node
    if n_mag != 0:
        Omega = np.degrees(np.arccos(n_vec[0] / n_mag))
        if n_vec[1] < 0:
            Omega = 360 - Omega
    else:
        Omega = 0

    # Argument of periapsis
    if n_mag != 0 and e > 1e-8:
        omega = np.degrees(np.arccos(np.dot(n_vec, e_vec) / (n_mag * e)))
        if e_vec[2] < 0:
            omega = 360 - omega
    else:
        omega = 0

    # True anomaly
    if e > 1e-8:
        nu = np.degrees(np.arccos(np.dot(e_vec, r_rel) / (e * r_mag)))
        if np.dot(r_rel, v_rel) < 0:
            nu = 360 - nu
    else:
        nu = 0

    # Orbital period (years)
    T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))

    print(f"Semi-major axis (pc): {a:.3f}, Eccentricity: {e:.3f}, Inclination (deg): {i_deg:.3f}")
    print(f"Longitude of ascending node (deg): {Omega:.3f}, Argument of periapsis (deg): {omega:.3f}, True anomaly (deg): {nu:.3f}")
    print(f"Orbital period (years): {T_yr:.3f}")
    print("")

    # Return the orbital elements as a dictionary
    return {
        'semi_major_axis_pc': a,
        'eccentricity': e,
        'inclination_deg': i_deg,
        'longitude_of_ascending_node_deg': Omega,
        'argument_of_periapsis_deg': omega,
        'true_anomaly_deg': nu,
        'orbital_period_yr': T_yr
    }

# Example stars (without radial velocity)
example_star1 = {
    'ra': 56.477085,
    'dec': 24.554305,
    'parallax': 7.305275,              # Distance ~133.3 pc
    'pmra': 20.331403,                 # mas/yr
    'pmdec': -46.022055,               # mas/yr
    'radial_velocity': 11.792832,               
    'mass': 3.47                       # Msun
}

example_star2 = {
    'ra': 56.479764,                 # ~3.6 arcsec east = ~0.0023 pc
    'dec': 24.554604,                # ~3.6 arcsec north = ~0.0023 pc
    'parallax': 7.355189,              # Same distance
    'pmra': 19.533205,                 # Slight relative tangential motion
    'pmdec': -45.550902,
    'radial_velocity': 4.299448,      # Slight relative radial velocity
    'mass': 0.58
}

# Compute orbital elements without radial velocity
full_orbit_params = compute_orbital_elements_from_phase_space_without_rv(example_star1, example_star2)


Computed energy total: 14.093
Semi-major axis (pc): -0.000, Eccentricity: 329.524, Inclination (deg): 43.760
Longitude of ascending node (deg): 264.517, Argument of periapsis (deg): 239.222, True anomaly (deg): 83.922
Orbital period (years): nan



C:\Users\bosca\AppData\Local\Temp\ipykernel_11428\380811587.py:95: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))


## Let's try with our data

In [24]:
#Gaia_df = pd.read_csv('DATA/My_Pleiades_filtered_Gaia_DR3_with_mass.csv')
binaries_candidates_df = pd.read_csv('DATA/My_Pleiades_binary_pairs.csv')

binaries_candidates_df.head(20)

,primary_source_id,secondary_source_id
0,66507469798631936,66507469798632320
1,68364544933829376,68364544935515392
2,64956123313498368,64956127609464320
3,66733552578791296,66733556873061120
4,65207709611941376,65207709613871744
5,66798496781121792,66798526845337344
6,65241313435901568,65241313437941504
7,65247704349267584,65248460263511552
8,65266494828710400,65266499126062080
9,65272817023559040,65272821318002560


In [25]:
# taking the binaries candidates from the Gaia DR3
#binaries_candidates_index1 = binaries_candidates_df['primary_source_id'].values
#binaries_candidates_index2 = binaries_candidates_df['secondary_source_id'].values

# Get the Gaia DR3 data for the binary candidates
binaries_1 = pd.read_csv('DATA/Chulkov_primary_members_binary_Pleiades_filtered.csv')
binaries_2 = pd.read_csv('DATA/Chulkov_secondary_members_binary_Pleiades_filtered.csv')

# orbits parameters
orbit_params = ['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'parallax', 'radial_velocity', 'mass', 'ra_error', 'dec_error', 'pmra_error', 'pmdec_error', 'parallax_error', 'radial_velocity_error', 'mass_error']

# Get the orbit parameters for the binary candidates
binaries_members_1 = binaries_1[orbit_params].copy()
binaries_members_2 = binaries_2[orbit_params].copy()

print(binaries_members_1.shape)
print(binaries_members_2.shape)

print('Check if the binaries candidates df contains Nan values:')
print(binaries_members_1.isnull().values.any())
print(binaries_members_2.isnull().values.any())
print(binaries_members_1.isna().values.any())
print(binaries_members_2.isna().values.any())

# Display the first few rows of the binaries members DataFrame

binaries_members_1.head()

(12, 15)
(12, 15)
Check if the binaries candidates df contains Nan values:
False
False
False
False


,source_id,ra,dec,pmra,pmdec,parallax,radial_velocity,mass,ra_error,dec_error,pmra_error,pmdec_error,parallax_error,radial_velocity_error,mass_error
0,66798496781121792,56.477085,24.554305,20.331403,-46.022055,7.305275,11.792832,3.47,0.072686,0.047801,0.094875,0.062754,0.083877,0.599952,0.15
1,64956127609464320,56.453495,23.146943,20.513910,-45.699760,7.421140,7.813674,2.45,0.039314,0.026078,0.052866,0.033643,0.041989,1.356579,0.12
2,65207709611941376,56.851821,23.914478,20.049108,-44.132590,7.220987,-1.856786,2.16,0.030519,0.022987,0.041599,0.029272,0.034383,2.441681,0.11
3,65272821318002560,56.098225,24.132439,28.070651,-43.097951,7.363603,6.376698,1.40,0.087245,0.067360,0.105531,0.081801,0.100038,0.419253,0.05
4,66507469798631936,57.491184,23.847947,21.026666,-48.132843,7.361154,4.707757,1.15,0.033703,0.018977,0.038665,0.025343,0.033577,0.533899,0.04


## Calculating the Bulk motion of the Pleiades

In [26]:
## first we remove all the nan values from the Gaia DataFrame:
#Gaia_df = Gaia_df.dropna(subset=orbit_params)

In [27]:
'''
# Weighted mean function
def weighted_mean_and_error(values, errors):
    weights = 1 / errors**2
    return np.sum(values * weights) / np.sum(weights), np.sqrt(1 / np.sum(weights))

# Compute weighted bulk motion and error
bulk_pmra, bulk_pmra_err = weighted_mean_and_error(Gaia_df['pmra'].values, Gaia_df['pmra_error'].values)
bulk_pmdec, bulk_pmdec_err = weighted_mean_and_error(Gaia_df['pmdec'].values, Gaia_df['pmdec_error'].values)
bulk_rv, bulk_rv_err = weighted_mean_and_error(Gaia_df['radial_velocity'].values, Gaia_df['radial_velocity_error'].values)

bulk_parallax, bulk_parallax_err = weighted_mean_and_error(Gaia_df['parallax'].values, Gaia_df['parallax_error'].values)

print(f"Bulk proper motion in RA: {bulk_pmra} ± {bulk_pmra_err} mas/yr")
print(f"Bulk proper motion in Dec: {bulk_pmdec} ± {bulk_pmdec_err} mas/yr")
print(f"Bulk radial velocity: {bulk_rv} ± {bulk_rv_err} km/s")
print(f"Bulk parallax: {bulk_parallax} ± {bulk_parallax_err} mas")
print(f"Number of stars used: {len(Gaia_df)}")
'''

'\n# Weighted mean function\ndef weighted_mean_and_error(values, errors):\n    weights = 1 / errors**2\n    return np.sum(values * weights) / np.sum(weights), np.sqrt(1 / np.sum(weights))\n\n# Compute weighted bulk motion and error\nbulk_pmra, bulk_pmra_err = weighted_mean_and_error(Gaia_df[\'pmra\'].values, Gaia_df[\'pmra_error\'].values)\nbulk_pmdec, bulk_pmdec_err = weighted_mean_and_error(Gaia_df[\'pmdec\'].values, Gaia_df[\'pmdec_error\'].values)\nbulk_rv, bulk_rv_err = weighted_mean_and_error(Gaia_df[\'radial_velocity\'].values, Gaia_df[\'radial_velocity_error\'].values)\n\nbulk_parallax, bulk_parallax_err = weighted_mean_and_error(Gaia_df[\'parallax\'].values, Gaia_df[\'parallax_error\'].values)\n\nprint(f"Bulk proper motion in RA: {bulk_pmra} ± {bulk_pmra_err} mas/yr")\nprint(f"Bulk proper motion in Dec: {bulk_pmdec} ± {bulk_pmdec_err} mas/yr")\nprint(f"Bulk radial velocity: {bulk_rv} ± {bulk_rv_err} km/s")\nprint(f"Bulk parallax: {bulk_parallax} ± {bulk_parallax_err} mas")\npr

## Monte Carlo sampling for the error

Since we have only one measurament we apply a MCM sampling to retrive the values of each orbit parameters and its error

| Parameter       | Range Chosen       | Why                                                                                 |
| --------------- | ------------------ | ----------------------------------------------------------------------------------- |
| `parallax`      | (0, 20) mas        | Pleiades parallax is \~7.4 mas. 20 mas is generous and avoids unphysical negatives. |
| `pmra`, `pmdec` | (-100, 100) mas/yr | Wide enough to include all reasonable proper motions for Gaia stars                 |
| `rv`            | (-100, 100) km/s   | Wide enough for almost all stellar RVs in the Galaxy                                |


In [28]:
import pandas as pd
import numpy as np
import emcee
from astropy.coordinates import SkyCoord
from astropy import units as u

# Define MCMC components
def log_prior(theta, obs, obs_err):
    ra, dec, plx, pmra, pmdec, rv, mass = theta
    if 0 < plx < 20 and -100 < pmra < 100 and -100 < pmdec < 100 and -100 < rv < 100 and 0 < mass < 10:
        # Check if the parameters are within the observed values and their errors
        if (np.abs(ra - obs[0]) < obs_err[0] and
            np.abs(dec - obs[1]) < obs_err[1] and
            np.abs(plx - obs[2]) < obs_err[2] and
            np.abs(pmra - obs[3]) < obs_err[3] and
            np.abs(pmdec - obs[4]) < obs_err[4] and
            np.abs(rv - obs[5]) < obs_err[5] and
            np.abs(mass - obs[6]) < obs_err[6]):
            return 0.0  # Log prior is zero for uniform prior within bounds
        return 0.0
    return -np.inf

def log_likelihood(theta, obs, obs_err):
    return -0.5 * np.sum(((theta - obs) / obs_err) ** 2)

def log_posterior(theta, obs, obs_err):
    lp = log_prior(theta, obs, obs_err)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, obs, obs_err)

def mcmc_sample_star(obs_vals, obs_errs, nwalkers=16, nsteps=300):
    ndim = len(obs_vals)
    p0 = obs_vals + 1e-4 * np.random.randn(nwalkers, ndim)
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(obs_vals, obs_errs))
    sampler.run_mcmc(p0, nsteps, progress=False)
    samples = sampler.get_chain(discard=int(nsteps * 0.2), thin=10, flat=True)
    return samples

def summarize_samples(samples):
    param_names = ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'radial_velocity', 'mass']
    summary = {}
    for i, name in enumerate(param_names):
        summary[f'{name}'] = np.mean(samples[:, i])
        summary[f'{name}_error'] = np.std(samples[:, i])
    return summary



def run_MCM_on_binaries(df, summary_list):
    for _, row in df.iterrows():
        obs = np.array([
            row['ra'], row['dec'], row['parallax'],
            row['pmra'], row['pmdec'], row['radial_velocity'], row['mass']
        ])
        obs_err = np.array([
            row['ra_error'], row['dec_error'], row['parallax_error'],
            row['pmra_error'], row['pmdec_error'], row['radial_velocity_error'], row['mass_error']
        ])
        samples = mcmc_sample_star(obs, obs_err, nwalkers=16, nsteps=1000)
        summary = summarize_samples(samples)
        summary_list.append(summary)
    return summary_list

# Create summary DataFrame both for binaries_members_1 and binaries_members_2
summary_list1 = []
summary_list1 = run_MCM_on_binaries(binaries_members_1, summary_list1)

summary_list2 = []
summary_list2 = run_MCM_on_binaries(binaries_members_2, summary_list2)

# Convert the summary list to a DataFrame
binaries_members_1_MCM = pd.DataFrame(summary_list1)
binaries_members_2_MCM = pd.DataFrame(summary_list2)

binaries_members_1_MCM.head(10)


c:\Users\bosca\anaconda3\Lib\site-packages\emcee\moves\red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]


,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,radial_velocity,radial_velocity_error,mass,mass_error
0,56.475736,0.078724,24.553344,0.046293,7.299805,0.086042,20.330231,0.092798,-46.017148,0.059345,11.851299,0.584343,3.467611,0.151243
1,56.453764,0.041166,23.147488,0.026165,7.415671,0.043486,20.517970,0.054460,-45.698804,0.031162,7.995394,1.332174,2.459329,0.118209
2,56.855833,0.030534,23.913015,0.023624,7.223422,0.036590,20.052894,0.039740,-44.132049,0.029466,-1.947241,2.457083,2.161588,0.105025
3,56.102880,0.081968,24.139942,0.066048,7.364426,0.101154,28.057108,0.105428,-43.099508,0.078973,6.310281,0.404513,1.404605,0.050487
4,57.485734,0.033498,23.848064,0.019063,7.361250,0.034450,21.028609,0.039727,-48.133291,0.023758,4.794476,0.504126,1.152142,0.039968
5,56.055109,0.018812,24.031424,0.011847,7.595631,0.020864,21.405934,0.021645,-45.884967,0.015577,4.184285,0.479555,0.931142,0.028197
6,56.358079,0.013592,23.430268,0.010615,5.958520,0.015876,16.154463,0.019667,-35.835357,0.014014,3.682942,2.226736,0.807705,0.021591
7,56.654302,0.065522,24.340715,0.042556,7.402784,0.059513,22.382437,0.113164,-45.517462,0.060115,6.943125,3.055027,0.740853,0.019972
8,58.306468,0.014989,23.929523,0.008034,8.424585,0.015298,10.315724,0.017522,-47.004515,0.010523,13.373828,0.763813,0.729554,0.021112
9,55.514506,0.169622,24.707496,0.161939,7.802670,0.187282,22.209790,0.236602,-46.627813,0.168406,5.552034,7.237141,0.608618,0.020171


In [29]:
## adding the corresponding source_id to the MCM results
binaries_members_1_MCM.insert(0, 'source_id', binaries_members_1['source_id'].values)
binaries_members_2_MCM.insert(0, 'source_id', binaries_members_2['source_id'].values)

binaries_members_2_MCM.head(10)

,source_id,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,radial_velocity,radial_velocity_error,mass,mass_error
0,66798526845337344,56.480530,0.022198,24.555395,0.014758,7.355737,0.024021,19.532466,0.032507,-45.550886,0.019753,4.674864,4.864223,0.579645,0.010126
1,64956123313498368,56.454175,0.019013,23.147792,0.012270,7.407750,0.018189,19.028539,0.023592,-45.463380,0.015225,5.293785,0.219137,1.129459,0.028498
2,65207709613871744,56.847885,0.044832,23.916552,0.035585,7.185950,0.054458,19.608511,0.059311,-46.336292,0.040675,5.370074,0.582369,1.092736,0.032673
3,65272817023559040,56.094990,0.029693,24.129470,0.023050,7.248982,0.036346,20.323159,0.038597,-47.782157,0.031122,10.449483,7.029927,0.559140,0.010540
4,66507469798632320,57.509816,0.245401,23.849509,0.136467,7.629822,0.241367,21.528668,0.266102,-43.568122,0.180532,0.581543,2.526197,1.102201,0.029052
5,65248460263511552,56.071761,0.107054,24.041065,0.093366,7.374164,0.137742,20.152351,0.118906,-45.159024,0.099567,4.637589,1.277787,0.819060,0.019728
6,65163797866480768,56.357492,0.000079,23.429463,0.000077,5.700737,0.000087,17.336957,0.000104,-37.249775,0.000103,100.350927,0.000097,0.420020,0.000138
7,66733556873061120,56.647150,0.080128,24.346031,0.056034,7.292024,0.074820,16.350362,0.145701,-44.763132,0.090977,3.990496,1.732419,0.708551,0.020692
8,66474450090139008,58.301329,0.035746,23.929714,0.021399,8.455221,0.037197,12.057863,0.041758,-46.304861,0.028254,13.847876,6.011725,0.450172,0.020066
9,68364544933829376,55.512624,0.044224,24.708832,0.035370,7.480271,0.058481,21.717353,0.067922,-45.295875,0.062228,11.509912,4.373608,0.600512,0.018445


## ok Now we calculate the orbits parameters for each pair

In [32]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

def compute_orbital_elements_from_phase_space(star1, star2):
    G = 4.302e-3  # pc * (km/s)^2 / Msun

    # Average parallax to enforce consistent distance scale
    avg_parallax = (star1['parallax'] + star2['parallax']) / 2  # mas
    distance = (1000 / avg_parallax) * u.pc

    # SkyCoord objects
    c1 = SkyCoord(
        ra=star1['ra'] * u.deg,
        dec=star1['dec'] * u.deg,
        distance=distance,
        #distance=(1000 / star1['parallax']) * u.pc,
        pm_ra_cosdec=star1['pmra'] * u.mas/u.yr,
        pm_dec=star1['pmdec'] * u.mas/u.yr,
        radial_velocity=star1['radial_velocity'] * u.km/u.s
    )

    c2 = SkyCoord(
        ra=star2['ra'] * u.deg,
        dec=star2['dec'] * u.deg,
        distance=distance,
        #distance=(1000 / star2['parallax']) * u.pc,
        pm_ra_cosdec=star2['pmra'] * u.mas/u.yr,
        pm_dec=star2['pmdec'] * u.mas/u.yr,
        radial_velocity=star2['radial_velocity'] * u.km/u.s
    )

    # Masses and CM quantities
    m1, m2 = star1['mass'], star2['mass']
    M = m1 + m2
    M_r = (m1 * m2) / M
    mu = G*M

    r1 = c1.cartesian.xyz.to(u.pc).value
    v1 = c1.velocity.d_xyz.to(u.km/u.s).value
    r2 = c2.cartesian.xyz.to(u.pc).value
    v2 = c2.velocity.d_xyz.to(u.km/u.s).value

    print(f"Computed positions: {r1}, {r2}")
    print(f"Computed velocities: {v1}, {v2}")

    # Relative position and velocity in CM frame
    r_rel = r2 - r1
    v_rel = v2 - v1
    r_mag = np.linalg.norm(r_rel)
    v_mag = np.linalg.norm(v_rel)

    # Angular momentum
    h_vec = np.cross(r_rel, v_rel)
    h_mag = np.linalg.norm(h_vec)

    # Specific mechanical energy
    E = 0.5*v_mag**2 - mu/r_mag

    E_tot = M_r*(0.5*v_mag**2. - mu/r_mag)
    a = - mu/(2 * E)  # Semi-major axis

    print(f"Computed specific energy: {E:.3f} and tot: {E_tot:.3f}")
    print(f"Computed semi-major axis tot: {a:.3f}")

    # Eccentricity vector
    e_vec = (np.cross(v_rel, h_vec) / mu) - (r_rel / r_mag) 
    e = np.linalg.norm(e_vec)

    print(f"Computed eccentricity tot: {e:.3f}")


    # Inclination
    i_rad = np.arccos(h_vec[2] / h_mag)
    i_deg = np.degrees(i_rad)

    # Node vector
    K = np.array([0, 0, 1])
    n_vec = np.cross(K, h_vec)
    n_mag = np.linalg.norm(n_vec)

    # Longitude of ascending node
    if n_mag != 0:
        Omega = np.degrees(np.arccos(n_vec[0] / n_mag))
        if n_vec[1] < 0:
            Omega = 360 - Omega
    else:
        Omega = 0

    # Argument of periapsis
    if n_mag != 0 and e > 1e-8:
        omega = np.degrees(np.arccos(np.dot(n_vec, e_vec) / (n_mag * e)))
        if e_vec[2] < 0:
            omega = 360 - omega
    else:
        omega = 0

    # True anomaly
    if e > 1e-8:
        nu = np.degrees(np.arccos(np.dot(e_vec, r_rel) / (e * r_mag)))
        if np.dot(r_rel, v_rel) < 0:
            nu = 360 - nu
    else:
        nu = 0

    # Orbital period (years)
    T_yr = 2 * np.pi * np.sqrt((a**3) / mu)


    print(f"Semi-major axis (pc): {a:.3f}, Eccentricity: {e:.3f}, Inclination (deg): {i_deg:.3f}")
    print(f"Longitude of ascending node (deg): {Omega:.3f}, Argument of periapsis (deg): {omega:.3f}, True anomaly (deg): {nu:.3f}")
    print(f"Orbital period (years): {T_yr:.3f}")
    print("")

    # Return the orbital elements as a dictionary

    return {
        'semi_major_axis_pc': a,
        'eccentricity': e,
        'inclination_deg': i_deg,
        'longitude_of_ascending_node_deg': Omega,
        'argument_of_periapsis_deg': omega,
        'true_anomaly_deg': nu,
        'orbital_period_yr': T_yr
    }




if not os.path.exists('results'):
    os.makedirs('results/')

result_orbital_elements_pair = {}
orbital_features = ['source_id', 'ra', 'dec', 'parallax', 'pmra', 'pmdec', 'radial_velocity', 'mass']

for i in range(len(binaries_members_1_MCM)):
    star1 = binaries_members_1_MCM.loc[i, orbital_features]
    star2 = binaries_members_2_MCM.loc[i, orbital_features]
        
    print(f"Computed orbital elements for the pair number {i}")

    # Compute the orbital elements from the phase space
    star1 = star1.to_dict()
    star2 = star2.to_dict()
    
    # Compute orbital elements
    orbital_elements = compute_orbital_elements_from_phase_space(star1, star2)
    
    result_orbital_elements_pair[i] = orbital_elements

# Convert the dictionary to a DataFrame
orbital_elements_df = pd.DataFrame.from_dict(result_orbital_elements_pair, orient='index')

if save:
    # Save the DataFrame to a CSV file
    orbital_elements_df.to_csv('results/Chulkov/Chulkov_orbital_elements_wRV.csv', index=False)

    # Save the results to CSV files
    #binaries_members_1_MCM.to_csv('results/Chulkov/Chulkov_binaries_members_1_MCM.csv', index=False)
    #binaries_members_2_MCM.to_csv('results/Chulkov/Chulkov_binaries_members_2_MCM.csv', index=False)

    print("Results saved to 'results/' directory.")


Computed orbital elements for the pair number 0
Computed positions: [ 68.5541525  103.47878611  56.70759384], [ 68.54437251 103.48282984  56.71203685]
Computed velocities: [  1.82132389  26.56276717 -22.15275227], [ -1.42398077  20.73220174 -24.85990123]
Computed specific energy: 24.411 and tot: 12.123
Computed semi-major axis tot: -0.000
Computed eccentricity tot: 33.148
Semi-major axis (pc): -0.000, Eccentricity: 33.148, Inclination (deg): 31.831
Longitude of ascending node (deg): 20.091, Argument of periapsis (deg): 135.543, True anomaly (deg): 357.238
Orbital period (years): nan

Computed orbital elements for the pair number 1
Computed positions: [ 68.55670715 103.39650093  53.0376045 ], [ 68.55580935 103.39675857  53.03826274]
Computed velocities: [ -0.52536205  22.95520873 -23.73258661], [ -1.13693455  20.30918151 -24.65604781]
Computed specific energy: -9.397 and tot: -7.273
Computed semi-major axis tot: 0.001
Computed eccentricity tot: 0.443
Semi-major axis (pc): 0.001, Eccentr

C:\Users\bosca\AppData\Local\Temp\ipykernel_11428\3807266446.py:107: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / mu)
C:\Users\bosca\AppData\Local\Temp\ipykernel_11428\3807266446.py:107: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / mu)
C:\Users\bosca\AppData\Local\Temp\ipykernel_11428\3807266446.py:107: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / mu)
C:\Users\bosca\AppData\Local\Temp\ipykernel_11428\3807266446.py:107: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / mu)
C:\Users\bosca\AppData\Local\Temp\ipykernel_11428\3807266446.py:107: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / mu)
C:\Users\bosca\AppData\Local\Temp\ipykernel_11428\3807266446.py:107: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / mu)
C:\Users\bosca\AppData\Local\Temp\

In [31]:
print(binaries_members_1_MCM[['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'radial_velocity', 'mass']].head(5))
print(binaries_members_2_MCM[['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'radial_velocity', 'mass']].head(5))

          ra        dec  parallax       pmra      pmdec  radial_velocity  \
0  56.475736  24.553344  7.299805  20.330231 -46.017148        11.851299   
1  56.453764  23.147488  7.415671  20.517970 -45.698804         7.995394   
2  56.855833  23.913015  7.223422  20.052894 -44.132049        -1.947241   
3  56.102880  24.139942  7.364426  28.057108 -43.099508         6.310281   
4  57.485734  23.848064  7.361250  21.028609 -48.133291         4.794476   

       mass  
0  3.467611  
1  2.459329  
2  2.161588  
3  1.404605  
4  1.152142  
          ra        dec  parallax       pmra      pmdec  radial_velocity  \
0  56.480530  24.555395  7.355737  19.532466 -45.550886         4.674864   
1  56.454175  23.147792  7.407750  19.028539 -45.463380         5.293785   
2  56.847885  23.916552  7.185950  19.608511 -46.336292         5.370074   
3  56.094990  24.129470  7.248982  20.323159 -47.782157        10.449483   
4  57.509816  23.849509  7.629822  21.528668 -43.568122         0.581543   

  